In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("data/retail_transactions.csv", parse_dates=["ds"])
df.head()

In [ ]:
# Plot series


In [ ]:
plot_series(df=df, max_insample_length=56, palette="viridis")

## Pretrain a model

In [ ]:
from neuralforecast.core import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.losses.pytorch import MSE

In [ ]:
# Pretrain on books category only


In [ ]:
# Set horizon to 28

# Define models list

# Initialize NeuralForecast object

# Fit the model


In [ ]:
# Save the model


## Transfer learning

In [ ]:
# Load the model


In [ ]:
# Select all categories except "Books" for forecasting


In [ ]:
# Split into input/test. Keep the last 28 time steps for the test set


### Zero-shot forecasting

In [ ]:
# Zero-shot forecasts

zero_shot_fcsts = zero_shot_fcsts.rename(columns={"PatchTST": "PatchTST_ZS"})
zero_shot_fcsts.head()

### Fine-tuning

In [ ]:
# Set the number of fine-tuning steps


In [ ]:
# Fit to fine-tune


In [ ]:
# Forecast with the fine-tuned model

finetuned_fcsts = finetuned_fcsts.rename(columns={"PatchTST": "PatchTST_FT"})
finetuned_fcsts.head()

### Data-specfic model

In [ ]:
horizon = 28

models = [
    PatchTST(
        h=horizon,
        input_size=2*horizon,
        loss=MSE(),
        valid_loss=MSE(),
        scaler_type="robust",
        early_stop_patience_steps=3,
        max_steps=1000
    )
]

nf = NeuralForecast(models=models, freq="D")
nf.fit(df=input_df, val_size=horizon)
fcsts = nf.predict()
fcsts.head()

## Evaluation

In [ ]:
test_df = pd.merge(test_df, zero_shot_fcsts, 'left', ["unique_id", "ds"])
test_df = pd.merge(test_df, finetuned_fcsts, 'left', ["unique_id", "ds"])
test_df = pd.merge(test_df, fcsts, 'left', ["unique_id", "ds"])

test_df = test_df.drop(columns=["average_discount", "holiday", "week_before_holiday"])
test_df.head()

In [ ]:
plot_series(df=test_df, forecasts_df=test_df.drop(columns="y"), palette="viridis")

In [ ]:
# Evaluate each approach
